[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_img_accessibility_a_i.ipynb)

In [ ]:
%pip install openai
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Convert images to an accessible PDF/A document
Convert images to an accessible PDF/A-2a document.
Alternative texts are generated using the OpenAI API with
a dedicated prompt.

In [ ]:
import io
from openai import OpenAI
import base64
from pdftools_sdk.image import Document as ImageDocument, DocumentList as ImageDocumentList
from pdftools_sdk.pdf import Conformance
from pdftools_sdk.image2_pdf import Converter
from pdftools_sdk.image2_pdf.profiles import Archive

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_urls = [
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/coffee.jpg',
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/connection.jpg',
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/butterfly.jpg',
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/DigitalSignature.jpg'
]
image_paths = ['coffee.jpg', 'connection.jpg', 'butterfly.jpg', 'DigitalSignature.jpg']
for url, path in zip(input_urls, image_paths):
    prepare_file(url, path)
output_file_path = 'AccessiblePDFA.pdf'

In [ ]:
def get_alternate_text(image_path: str):
    # Getting base64 representation of input image
    with io.FileIO(image_path, 'rb') as image_stream:
        base64_image = base64.b64encode(image_stream.read()).decode('utf-8')

    # Instantiate OpenAI client and let AI create the alternate text
    client = OpenAI(api_key="***insert-open-ai-api-key***")
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
            "role": "user",
            "content": [
                {"type": "text", "text": "Write a short sentence what can be seen on the image. It shall explain to a person with impaired vision what is on the image. Write the answer in a poetic way in english."},
                {
                    "type": "image_url",
                    "image_url":
                        {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                },
            ],
            }
        ],
        max_tokens=300,
    )

    return response.choices[0].message.content.strip()

In [ ]:
def images_to_accessible_pdf(image_paths: list[str], output_file_path: str):
    # Store stream descriptors and images in lists
    stream_list = []
    images = ImageDocumentList()

    # Loop over all the image paths and store opened images into list
    for input_image_path in image_paths:
        image_stream = io.FileIO(input_image_path, 'rb')
        stream_list.append(image_stream)
        images.append(ImageDocument.open(image_stream))

    # Create output stream for writing
    with io.FileIO(output_file_path, 'wb+') as output_stream:
        # Create the profile that defines the conversion parameters.
        # The Archive profile converts images to PDF/A documents for archiving.
        profile = Archive()

        # Set conformance of output document to PDF/A-2a
        profile.conformance = Conformance.PDF_A2_A

        # For PDF/A level A, an alternate text is required for each page of the image.
        # This is optional for other PDF/A levels, e.g. PDF/A-2b.
        profile.language = "en"

        # Set alternate texts created by AI
        alternate_text_list = profile.alternate_text
        for image_path in image_paths:                
            alternate_text = get_alternate_text(image_path)
            alternate_text_list.append(alternate_text)

        converter = Converter()
        out_document = converter.convert_multiple(images, output_stream, profile)
        if not out_document:
            print(f"Error while converting images to Pdf.")
            return

    # Cleanup in finally block
    for image in images:
        image.__exit__(None, None, None)
    for stream in stream_list:
        stream.__exit__(None, None, None)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("INSERT-LICENSE-KEY")

    images_to_accessible_pdf(image_paths, output_file_path)

    print(f"Successfully created file {output_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")